In [1]:
pwd

'/home/dsc/Downloads'

In [2]:
cd ..


/home/dsc


In [3]:
cd ..


/home


In [21]:
!find -maxdepth 3 -type d -iname 'master*'

./dsc/Repositories/master-data-science


In [22]:
cd ./dsc/Repositories/master-data-science

/home/dsc/Repositories/master-data-science


In [23]:
#Este archivo tiene cambios. deberían visualizarse en la web github

Hist_cmd/  practice/  README.md


In [1]:
!git add .


In [4]:
!git push


Counting objects: 6, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 1.12 KiB | 286.00 KiB/s, done.
Total 6 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/hguadalupe/master-data-science.git
   c8305bf..c5d59df  master -> master


In [5]:
cd ..

/home/dsc/Repositories/master-data-science/practice


In [8]:
!find ../../../ -maxdepth 2 -iname 'data' 

../../../Data


In [12]:
arr_route = '../../../Data/challenge'

In [14]:
!find '../../../Data/challenge' -maxdepth 4 -type f -iname 'book*' 

../../../Data/challenge/bookings.csv.bz2


In [16]:
!bzip2 -dk ../../../Data/challenge/bookings.csv.bz2


bzip2: Can't open input file ../../../Data/challenge!ls/bookings.csv.bz2: No such file or directory.


In [6]:
!ls ../../../../Data/challenge

bookings.csv  bookings.csv.bz2	searches.csv.bz2


In [10]:
!head ../../../../Data/challenge/bookings.csv

head: cannot open '2' for reading: No such file or directory
==> ../../../../Data/challenge/bookings.csv <==
act_date           ^source^pos_ctry^pos_iata^pos_oid  ^rloc          ^cre_date           ^duration^distance^dep_port^dep_city^dep_ctry^arr_port^arr_city^arr_ctry^lst_port^lst_city^lst_ctry^brd_port^brd_city^brd_ctry^off_port^off_city^off_ctry^mkt_port^mkt_city^mkt_ctry^intl^route          ^carrier^bkg_class^cab_class^brd_time           ^off_time           ^pax^year^month^oid      
2013-03-05 00:00:00^1A    ^DE      ^a68dd7ae953c8acfb187a1af2dcbe123^1a11ae49fcbf545fd2afc1a24d88d2b7^ea65900e72d71f4626378e2ebd298267^2013-02-22 00:00:00^1708^0^ZRH     ^ZRH     ^CH      ^LHR     ^LON     ^GB      ^ZRH     ^ZRH     ^CH      ^LHR     ^LON     ^GB      ^ZRH     ^ZRH     ^CH      ^LHRZRH  ^LONZRH  ^CHGB    ^1^LHRZRH         ^VI^T        ^Y        ^2013-03-07 08:50:00^2013-03-07 11:33:37^-1^2013^3^NULL     
2013-03-26 00:00:00^1A    ^US      ^e612b9eeeee6f17f42d9b0d3b79e75ca^7437560d8f276

In [2]:
import pandas as pd
import numpy as np
import csv
from sqlalchemy import create_engine

In [25]:
#Ejercicio 1. Recuento de líneas (versión linux)
!wc -l ../../../../Data/challenge/bookings.csv 

10000011 ../../../../Data/challenge/bookings.csv


In [4]:
#Ejercicio 1. Recuento de líneas (versión python)
route = '../../../../Data/challenge/bookings.csv'
csv_opening = open(route)
count = sum(1 for row in csv_opening)
count

10000011

In [5]:
#Ejercicio 1. Tantea el archivo con cuidado, para ver contenido y tamaño.
pd_opening = pd.read_csv(route, sep='^',skip_blank_lines=True,nrows=10)
!ls -sh ../../../../Data/challenge/bookings.csv
print(list(pd_opening))
print(pd_opening)
#pd_opening = pd.read_csv('../../../../Data/challenge/bookings.csv', sep='^',skip_blank_lines=True)


4.0G ../../../../Data/challenge/bookings.csv
['act_date           ^source^pos_ctry^pos_iata^pos_oid  ^rloc          ^cre_date           ^duration^distance^dep_port^dep_city^dep_ctry^arr_port^arr_city^arr_ctry^lst_port^lst_city^lst_ctry^brd_port^brd_city^brd_ctry^off_port^off_city^off_ctry^mkt_port^mkt_city^mkt_ctry^intl^route          ^carrier^bkg_class^cab_class^brd_time           ^off_time           ^pax^year^month^oid      ']
  act_date           ^source^pos_ctry^pos_iata^pos_oid  ^rloc          ^cre_date           ^duration^distance^dep_port^dep_city^dep_ctry^arr_port^arr_city^arr_ctry^lst_port^lst_city^lst_ctry^brd_port^brd_city^brd_ctry^off_port^off_city^off_ctry^mkt_port^mkt_city^mkt_ctry^intl^route          ^carrier^bkg_class^cab_class^brd_time           ^off_time           ^pax^year^month^oid      
0  2013-03-05 00:00:00^1A    ^DE      ^a68dd7ae95...                                                                                                                                 

In [59]:
#Tenemos un archivo de 4Gigas y 10 millones de filas. VAmos a tener que dividirlo en partes. 
#Podemos dividirlo en "chunks" de 100000 líneas con sqlalchemy, iterando sobre el CSV.
#para ello tendremos que crear una bse de datos CSV.

with open('bigcsv.csv', 'wb') as csvfile:
    csvwriter =csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)

In [ ]:
#Nofuncionó:
def get_fromaction_data(fname,chunk_size=100000):
    reader = pd.read_csv(fname,chunksize=chunk_size,sep='^',skip_blank_lines=True, iterator=True)
    chunks =[]
    loop =True
    while loop:
        try:
            chunk =reader.get_chunk(chunk_size)
            chunks.append(chunk)
        except StopIteration:
            loop =False
            print("Iteration is stopped")
    
    df_ac =pd.concat(chunks,ignore_index=True)
    
get_fromaction_data(route)

/home/dsc/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (7,8,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Iteration is stopped


In [60]:
#Tenemos un archivo de 4Gigas y 10 millones de filas. VAmos a tener que dividirlo en partes. 
#Podemos dividirlo en "chunks" de 100000 líneas con sqlalchemy, iterando sobre el CSV.
chunksize = 100000
i = 0
j = 1
for df1 in pd.read_csv(route,chunksize=chunksize,sep='^',skip_blank_lines=True, iterator=True):
    df1.index += j
    i+=1
    df1.to_sql('table', csvfile, if_exists='append')
    j = df1.index[-1] + 1

#CAUTION df_top10= pd.read_csv(route, sep='^',skip_blank_lines=True).loc[:,'arr_port'].sum()

#df.columns.str.replace('\s+','')

/home/dsc/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:1362: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


AttributeError: '_io.BufferedWriter' object has no attribute 'cursor'

4.0G ../../../../Data/challenge/bookings.csv
